In [1]:
# Author: Mishma Mariyam Raju
# Created on July 12 10:15 am

In [2]:
import struct
import numpy as np
import math
from scipy.optimize import fsolve
import random
import os

In [3]:
def solve_for_G(mu):
    func = lambda G : G - mu * np.exp(G - mu)
    
    '''
    G = np.linspace(0.000, 1.000, 1000)
    plt.plot(G, func(G))
    plt.xlabel("G")
    plt.ylabel("expression value")
    plt.grid()
    plt.show()
    '''

    G_initial_guess = 0.5
    G_soln = fsolve(func, G_initial_guess)
    #print("The solution is G = %f" % G_soln)
    #print("at which the value of the expression is %f" % func(G_soln))
    return G_soln[0]

In [4]:
def ln_stirling_2nd(n, k):
    mu = n/k
    G = solve_for_G(mu)
    #G = 0.5
    return 0.5 * (math.log(mu - 1) - math.log(mu * (1 - G))) + \
        (n - k) * (math.log(mu - 1) - math.log(mu - G)) + \
            n * math.log(k) - k * math.log(n) + k * (1 - G) + \
                ln_n_fact(n) - ln_n_fact(n - k) - ln_n_fact(k)

In [5]:
def ln_n_fact(n):
    if(n > 8):
        return 0.5 * math.log(2*math.pi) - n + (n + 0.5) * math.log(n)
    else: 
        return math.log(math.factorial(n))

In [6]:
def P(x, n, l):
    if(x == int(l/n) or x == 1):
        return math.e ** (ln_n_fact(2 ** n) - ln_n_fact(2**n - x) + ((l%n) - l) * math.log(2))
    else:
        Term_1 = ln_n_fact(2 ** n) - ln_n_fact(2**n - x) 
        #Term_2 = ln_n_fact(int(l/n)) - ln_n_fact(int(l/n) - x) + x * math.log(int(l/n))
        Term_2 = ln_stirling_2nd(int(l/n), x)
        #Term_2 = math.log(stirling(int(l/n), x))
        Term_3 = ((l%n) - l) * math.log(2)
        #print(Term_1, Term_2, Term_3)
        #print(math.e ** (Term_1 + Term_2 + Term_3))
        return math.e ** (Term_1 + Term_2 + Term_3)

In [7]:
def E(n, l):
    E = 0.0
    sump = 0.0
    p = 0.0
    for x in range(1, min(2 ** n + 1, int(l/n)+1)):
        p = P(x, n, l)
        sump += p
        E += p * x
    '''
    p = [P(x, n, l) for x in range(1, min(2 ** n + 1, int(l/n)+1))]
    #print(p)
    E_of_n_l = [p[x-1] * x for x in range(1, min(2 ** n + 1, int(l/n)+1))]
    print("E(", n, "\b,", l, "\b)", int(sum(E_of_n_l)))
    '''
    #print(sump)
    #print("E(", n, "\b,", l, "\b) = ", round(E))
    return E

In [8]:
def find_length_of_bit_file(bit_file_name):
    size_in_bytes = os.path.getsize(bit_file_name)
    size_of_one_character = 1.0
    return int(size_in_bytes/size_of_one_character)

In [9]:
def find_dp(bit_file_name, n, offset = 0):
    corpus = open(bit_file_name)
    file = open("distinct_patterns_entropy_based_.txt", "w")
    for i in range(offset, n + offset):
        file.write(corpus.read(1))
    file.write("\n")
    file.close()
    
    length_of_bit_file = find_length_of_bit_file(bit_file_name)
    no_of_distinct_patterns = 1
    count_of_distinct_patterns = [1]
    
    for i in range(offset + n, length_of_bit_file - n, n):
        flag = 1
        pattern = []
        for j in range(0, n):
            pattern.append(int(corpus.read(1)))
        #print(pattern)
        with open('distinct_patterns_entropy_based_.txt') as f:
            for index, line in enumerate(f):
                #print("Line {}: {}".format(index, line.strip()))
                j = []
                for ch in line.strip():
                    j.append(int(ch))
                    pass
                if(np.array_equal(pattern, j)):
                    count_of_distinct_patterns[index] += 1
                    flag = 0
        file = open("distinct_patterns_entropy_based_.txt", "a")  
        if(flag == 1):
            for num in pattern:
                file.write(str(num))
            file.write("\n")
            no_of_distinct_patterns += 1
            count_of_distinct_patterns.append(1)
        file.close()
    print(no_of_distinct_patterns, count_of_distinct_patterns)
    return count_of_distinct_patterns

In [10]:
def entropy(arr):
    s = 0
    for i in arr:
      if i != 0:
        #print(-i / sum(arr)*math.log(i/sum(arr)))
        s += -i / sum(arr) * math.log2(i/sum(arr))
    print("Sum is ", s, "for ", len(arr), "\n")
    return s

In [11]:
def CS(bit_file_name, cs_file, offset = 0):
    file = open(cs_file, 'w')
    for n in range(1, 45):
        count_of_distinct_patterns = find_dp(bit_file_name, n, offset)
        E_no_of_distinct_patterns = E(n, find_length_of_bit_file(bit_file_name))
        print("ln E is ", math.log(E_no_of_distinct_patterns), "for E ", E_no_of_distinct_patterns)
        file.write(str(1 - entropy(count_of_distinct_patterns)/math.log2(E_no_of_distinct_patterns)) + ",") 
        #file.write(str(1 - len(count_of_distinct_patterns)/E(n, find_length_of_bit_file(bit_file_name))) + ",")  
    file.close()
    return

In [16]:
cs_file = 'CSs/cs_entropy_bit_born_a_crime_ASCII_256.txt'
bit_file_name = 'Corpuses/bit_born_a_crime_ASCII_256.txt'

CS(bit_file_name, cs_file)

2 [115350, 95273]
ln E is  0.6931471806763606 for E  2.0000000002328306
Sum is  0.9934356734056908 for  2 

4 [35443, 26182, 16144, 27542]
ln E is  1.3862943610034753 for E  3.9999999995343387
Sum is  1.9487898054736263 for  4 

8 [8274, 9492, 9511, 4355, 8297, 9598, 9629, 11051]
ln E is  2.0794415415634204 for E  7.999999999068677
Sum is  2.9628964167222045 for  8 

16 [3349, 1910, 13976, 1585, 607, 7320, 3233, 1929, 6226, 5218, 1727, 1592, 1106, 894, 1679, 304]
ln E is  2.7725887221815735 for E  15.999999999068677
Sum is  3.4056889448997953 for  16 

32 [850, 1811, 1487, 1513, 1164, 1833, 1325, 1291, 1358, 1049, 1885, 1182, 1092, 1102, 1044, 1839, 2024, 1545, 1720, 793, 1755, 416, 1993, 1803, 1768, 1598, 1944, 695, 1087, 740, 404, 14]
ln E is  3.465735902857934 for E  32.000000001862645
Sum is  4.865158461553292 for  32 

63 [400, 920, 441, 231, 1513, 741, 578, 690, 1442, 1044, 749, 362, 1465, 1701, 1249, 604, 496, 686, 1472, 241, 365, 154, 1316, 520, 517, 295, 864, 753, 421, 1029, 9

616 [14, 40, 208, 69, 121, 89, 661, 124, 34, 57, 4, 211, 43, 14, 39, 48, 12, 264, 518, 197, 8, 203, 90, 304, 10, 274, 188, 64, 215, 1, 16, 19, 59, 69, 171, 76, 124, 169, 192, 244, 141, 254, 56, 16, 24, 7, 16, 20, 228, 55, 59, 59, 164, 57, 170, 17, 31, 161, 122, 38, 72, 216, 8, 67, 41, 23, 61, 44, 22, 65, 83, 109, 179, 32, 42, 37, 53, 101, 34, 312, 52, 97, 44, 105, 18, 282, 281, 60, 12, 19, 39, 136, 56, 97, 5, 3, 61, 88, 40, 15, 86, 28, 65, 12, 87, 36, 35, 35, 30, 65, 147, 64, 8, 17, 7, 68, 39, 66, 7, 66, 29, 41, 4, 37, 66, 59, 77, 50, 13, 131, 97, 49, 88, 23, 9, 47, 13, 19, 24, 107, 17, 71, 83, 29, 21, 46, 78, 18, 30, 81, 11, 6, 18, 53, 75, 32, 143, 5, 35, 9, 32, 48, 16, 264, 25, 74, 23, 75, 5, 28, 19, 3, 20, 27, 12, 69, 20, 5, 5, 9, 107, 34, 58, 21, 24, 25, 41, 23, 8, 22, 11, 25, 72, 55, 17, 1, 10, 10, 2, 55, 26, 22, 2, 22, 13, 2, 28, 16, 4, 44, 11, 41, 10, 24, 19, 9, 56, 4, 1, 12, 27, 9, 16, 18, 9, 59, 44, 7, 18, 4, 16, 41, 49, 13, 9, 38, 29, 24, 6, 16, 15, 22, 23, 36, 4, 19, 5, 11, 

ln E is  8.862059789239755 for E  7059.007848513159
Sum is  10.088079585852613 for  2080 

1650 [3, 13, 8, 12, 63, 79, 12, 24, 25, 8, 11, 25, 1, 23, 7, 6, 56, 32, 13, 88, 79, 1, 1, 44, 28, 14, 103, 66, 35, 32, 5, 1, 26, 30, 28, 100, 21, 14, 16, 47, 48, 31, 23, 55, 10, 5, 2, 3, 5, 33, 34, 8, 16, 35, 72, 11, 18, 35, 8, 21, 93, 10, 6, 107, 23, 3, 7, 103, 9, 7, 61, 21, 18, 5, 34, 25, 71, 72, 27, 12, 12, 4, 13, 21, 13, 27, 15, 80, 11, 32, 4, 26, 22, 146, 24, 15, 40, 28, 3, 52, 41, 2, 8, 85, 110, 2, 19, 4, 4, 20, 9, 15, 56, 16, 1, 31, 11, 11, 94, 67, 67, 5, 10, 17, 16, 42, 8, 24, 24, 11, 45, 71, 21, 27, 36, 2, 3, 8, 13, 1, 18, 11, 1, 6, 42, 31, 6, 4, 38, 22, 15, 17, 66, 81, 21, 16, 31, 27, 9, 16, 9, 52, 15, 14, 6, 98, 32, 39, 2, 2, 13, 4, 15, 17, 67, 10, 73, 26, 8, 15, 17, 44, 47, 2, 27, 94, 34, 40, 12, 25, 6, 2, 17, 8, 4, 124, 29, 6, 37, 34, 23, 2, 7, 13, 11, 1, 14, 3, 35, 68, 28, 52, 2, 5, 26, 13, 8, 25, 7, 26, 72, 28, 26, 2, 19, 6, 35, 16, 9, 4, 22, 4, 6, 5, 8, 5, 24, 7, 5, 7, 42, 22, 14,

ln E is  9.343174922580845 for E  11420.610293921745
Sum is  10.829953729798293 for  3137 

695 [6, 77, 47, 191, 156, 74, 25, 4, 30, 35, 32, 19, 27, 17, 14, 250, 19, 4, 32, 100, 86, 253, 253, 58, 106, 30, 1, 25, 115, 26, 58, 40, 145, 354, 23, 14, 6, 3, 4, 3, 108, 89, 20, 13, 15, 32, 118, 13, 68, 211, 61, 49, 12, 36, 58, 72, 8, 44, 33, 219, 165, 53, 29, 62, 7, 15, 113, 12, 10, 16, 132, 56, 53, 35, 15, 4, 65, 12, 113, 50, 32, 76, 209, 95, 26, 14, 48, 5, 13, 48, 86, 86, 11, 21, 33, 17, 34, 9, 27, 125, 1, 13, 10, 47, 15, 43, 46, 25, 107, 66, 76, 64, 197, 39, 67, 40, 39, 37, 30, 31, 45, 23, 49, 125, 19, 14, 12, 23, 99, 29, 12, 9, 9, 72, 111, 9, 19, 13, 35, 62, 51, 51, 28, 83, 22, 99, 38, 36, 31, 9, 25, 19, 111, 14, 4, 32, 24, 22, 9, 121, 25, 16, 2, 29, 5, 44, 81, 7, 6, 10, 36, 13, 25, 5, 8, 34, 36, 7, 29, 40, 19, 68, 17, 38, 27, 8, 2, 13, 1, 100, 13, 33, 18, 2, 2, 16, 49, 12, 1, 3, 8, 6, 35, 45, 2, 32, 17, 88, 72, 11, 6, 32, 8, 22, 40, 19, 1, 27, 78, 13, 12, 15, 6, 17, 22, 8, 69, 10, 14, 1,

ln E is  9.377827638087147 for E  11823.30234427054
Sum is  11.403855352334473 for  4229 

3348 [1, 2, 14, 2, 20, 15, 5, 1, 11, 1, 2, 1, 13, 3, 7, 39, 1, 1, 5, 2, 71, 42, 5, 5, 12, 1, 5, 5, 1, 5, 28, 10, 13, 16, 12, 1, 2, 2, 1, 1, 8, 7, 3, 45, 3, 7, 7, 3, 6, 2, 5, 4, 8, 6, 5, 7, 27, 3, 9, 11, 19, 1, 14, 20, 1, 29, 30, 19, 3, 2, 7, 8, 7, 8, 15, 48, 17, 53, 1, 2, 7, 65, 13, 6, 3, 4, 19, 9, 3, 5, 3, 25, 2, 9, 2, 2, 7, 10, 12, 3, 35, 9, 2, 14, 55, 26, 6, 5, 1, 5, 5, 7, 2, 5, 1, 14, 9, 5, 16, 14, 51, 7, 6, 6, 7, 10, 4, 7, 3, 2, 2, 1, 2, 27, 5, 5, 6, 7, 6, 5, 2, 2, 33, 8, 6, 2, 11, 3, 10, 15, 13, 4, 3, 28, 1, 2, 3, 6, 6, 4, 2, 1, 54, 12, 8, 7, 8, 4, 13, 6, 3, 10, 8, 12, 20, 2, 13, 6, 28, 2, 8, 24, 4, 5, 2, 9, 25, 7, 6, 7, 5, 7, 10, 5, 5, 5, 6, 4, 1, 4, 18, 2, 6, 7, 12, 15, 1, 4, 13, 3, 2, 3, 4, 7, 9, 2, 4, 6, 4, 9, 8, 15, 19, 12, 1, 2, 6, 7, 9, 10, 1, 3, 13, 5, 28, 6, 21, 1, 25, 12, 5, 4, 1, 1, 3, 4, 11, 5, 3, 7, 4, 8, 5, 8, 1, 4, 1, 2, 3, 5, 1, 4, 4, 3, 5, 5, 3, 5, 3, 2, 29, 46, 40, 1, 5, 3

...

ln E is  8.569498092491315 for E  5268.484834390287
Sum is  11.796387718847015 for  4136 

4925 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

ln E is  8.54262756415032 for E  5128.80293099348
Sum is  12.23593790175307 for  4925 

4716 [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 4, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 2, 1, 1, 2, 1, 1, 1, 

ln E is  8.52590738596457 for E  5043.761368354518
Sum is  12.162665530139742 for  4716 

4769 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1

ln E is  8.496297138584833 for E  4896.603784493407
Sum is  12.203507073361525 for  4769 

4346 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 3, 1, 6, 2, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 3, 3, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

ln E is  8.455079882707633 for E  4698.881979541163
Sum is  12.012529266379481 for  4346 

